In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader

In [2]:
EPOCHS = 200
BATCH_SIZE = 128
LR = 0.01

# Data Preprocessing

In [3]:
badminton_data = pd.read_csv('./dataset.csv')
badminton_data = badminton_data[['rally', 'player', 'type', 'hit_height', 'landing_area', 'player_location_area','time_proportion']]
le = LabelEncoder()
badminton_data['type'] = le.fit_transform(badminton_data['type'])
training_data = badminton_data[:10789].reset_index(drop=True)
testing_data = badminton_data[10789:].reset_index(drop=True)

In [4]:
class BadmintonDataset(Dataset):
    def __init__(self, data):
        super(BadmintonDataset, self).__init__()
        rally = []
        mem = data['rally'][0]
        tmp = []
        for i in range(len(data)):
            if i == len(data)-1:
                rally.append(tmp)
            if data['rally'][i] != mem:
                mem = data['rally'][i+1]
                rally.append(tmp)
                tmp = []
            tmp.append([data['player'][i], data['type'][i], data['hit_height'][i], data['landing_area'][i], data['player_location_area'][i], data['time_proportion'][i]])

        self.pattern = []
        self.label = []
        for i in range(len(rally)):
            A = []
            B = []
            for j in range(len(rally[i])):
                if rally[i][j][0] == 'CHOU Tien Chen':
                    A.append(np.concatenate((np.array(rally[i][j][1]) * np.ones(1), np.array(rally[i][j][2]) * np.ones(1), np.array(rally[i][j][3]) * np.ones(1), np.array(rally[i][j][4]) * np.ones(1), np.array(rally[i][j][5]) * np.ones(1))))
                else:
                    B.append(np.concatenate((np.array(rally[i][j][1]) * np.ones(1), np.array(rally[i][j][2]) * np.ones(1), np.array(rally[i][j][3]) * np.ones(1), np.array(rally[i][j][4]) * np.ones(1), np.array(rally[i][j][5]) * np.ones(1))))
            if len(A) == 0 or len(B) == 0:
                continue
            self.pattern.append(np.concatenate((np.array(A), np.full((50 - len(A), 5), -10))))
            self.label.append(np.array([0]))
            self.pattern.append(np.concatenate((np.array(B), np.full((50 - len(B), 5), -10))))
            self.label.append(np.array([1]))
        self.label = np.array(self.label)
        self.pattern = np.array(self.pattern)
                
    def __len__(self):
        return len(self.label)

    def __getitem__(self, index):
        return self.pattern[index], self.label[index]

In [5]:
train_set = BadmintonDataset(badminton_data)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)

In [6]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(250, 128),
            nn.Tanh(),
            nn.Linear(128, 64),
            nn.Tanh(),
            nn.Linear(64, 16),
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(16, 64),
            nn.Tanh(),
            nn.Linear(64, 128),
            nn.Tanh(),
            nn.Linear(128, 250),
            nn.Sigmoid()
        )

    def forward(self, inputs):
        codes = self.encoder(inputs)
        decoded = self.decoder(codes)
        return codes, decoded

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
autoencoder_model = AutoEncoder()
optimizer = torch.optim.Adam(autoencoder_model.parameters(), lr=LR)
loss_function = nn.MSELoss()

autoencoder_model.to(device)
loss_function.to(device)

# Train
for epoch in range(EPOCHS):
    for data, labels in train_loader:
        inputs = data.view(-1, 250)
        inputs = inputs.to(device).float()
        # Forward
        codes, decoded = autoencoder_model(inputs)

        # Backward
        optimizer.zero_grad()
        loss = loss_function(decoded, inputs)
        loss.backward()
        optimizer.step()

    # Show progress
    print('[{}/{}] Loss:'.format(epoch+1, EPOCHS), loss.item())

# Save
torch.save(autoencoder_model, 'autoencoder.pt')

[1/200] Loss: 92.96614837646484
[2/200] Loss: 92.63709259033203
[3/200] Loss: 92.15216064453125
[4/200] Loss: 92.95531463623047
[5/200] Loss: 91.78447723388672
[6/200] Loss: 92.12538146972656
[7/200] Loss: 92.52552032470703
[8/200] Loss: 91.74050903320312
[9/200] Loss: 92.91446685791016
[10/200] Loss: 92.33477020263672
[11/200] Loss: 91.59191131591797
[12/200] Loss: 91.61124420166016
[13/200] Loss: 92.20622253417969
[14/200] Loss: 91.15541076660156
[15/200] Loss: 92.34761810302734
[16/200] Loss: 92.90929412841797
[17/200] Loss: 93.52404022216797
[18/200] Loss: 92.68351745605469
[19/200] Loss: 92.12432098388672
[20/200] Loss: 91.64353942871094
[21/200] Loss: 92.20222473144531
[22/200] Loss: 92.48206329345703
[23/200] Loss: 92.02784729003906
[24/200] Loss: 92.7489013671875
[25/200] Loss: 92.60089874267578
[26/200] Loss: 92.16314697265625
[27/200] Loss: 92.05864715576172
[28/200] Loss: 92.53560638427734
[29/200] Loss: 92.39261627197266
[30/200] Loss: 92.37794494628906
[31/200] Loss: 92.29

In [8]:
autoencoder_model = torch.load('autoencoder.pt')
autoencoder_model.eval()

AutoEncoder(
  (encoder): Sequential(
    (0): Linear(in_features=250, out_features=128, bias=True)
    (1): Tanh()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): Tanh()
    (4): Linear(in_features=64, out_features=16, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=16, out_features=64, bias=True)
    (1): Tanh()
    (2): Linear(in_features=64, out_features=128, bias=True)
    (3): Tanh()
    (4): Linear(in_features=128, out_features=250, bias=True)
    (5): Sigmoid()
  )
)

In [9]:
class FFN(nn.Module):
    def __init__(self, state_size=16):
        super(FFN, self).__init__()

        self.lr1 = nn.Linear(state_size, state_size)
        self.relu = nn.ReLU()
        self.lr2 = nn.Linear(state_size, state_size)
        self.dropout = nn.Dropout(0.2)
        self.pred = nn.Linear(state_size, 1)     
    
    def forward(self, x):
        x = self.lr1(x)
        x = self.relu(x)
        x = self.lr2(x)
        x = self.dropout(x)
        x = self.pred(x)
        return x

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ffn_model = FFN(16)
optimizer = torch.optim.Adam(ffn_model.parameters(), lr=LR)
loss_function = nn.BCEWithLogitsLoss()

ffn_model.to(device)
loss_function.to(device)

BCEWithLogitsLoss()

In [11]:
for epoch in range(EPOCHS):
    for data, label in train_loader:
        inputs = data.view(-1, 250)
        inputs = inputs.to(device).float()
        label = label.to(device).float()
        # Forward
        codes, decoded = autoencoder_model(inputs)
        output = ffn_model(codes)
        optimizer.zero_grad()
        loss = loss_function(output, label)
        loss.backward()
        optimizer.step()
    print('[{}/{}] Loss:'.format(epoch+1, EPOCHS), loss.item())
torch.save(ffn_model, 'ffn.pt')

[1/200] Loss: 0.6901876926422119
[2/200] Loss: 0.6980188488960266
[3/200] Loss: 0.6945430636405945
[4/200] Loss: 0.6961071491241455
[5/200] Loss: 0.6925288438796997
[6/200] Loss: 0.6934583783149719
[7/200] Loss: 0.695493221282959
[8/200] Loss: 0.692719578742981
[9/200] Loss: 0.6940052509307861
[10/200] Loss: 0.6944000720977783
[11/200] Loss: 0.6957583427429199
[12/200] Loss: 0.6920022964477539
[13/200] Loss: 0.6929763555526733
[14/200] Loss: 0.6914073824882507
[15/200] Loss: 0.693361222743988
[16/200] Loss: 0.6927630305290222
[17/200] Loss: 0.6958842873573303
[18/200] Loss: 0.6930668354034424
[19/200] Loss: 0.6938624978065491
[20/200] Loss: 0.6934561729431152
[21/200] Loss: 0.6935862302780151
[22/200] Loss: 0.6928880214691162
[23/200] Loss: 0.6940085887908936
[24/200] Loss: 0.6938445568084717
[25/200] Loss: 0.6923574209213257
[26/200] Loss: 0.6941049695014954
[27/200] Loss: 0.6932421922683716
[28/200] Loss: 0.6923415064811707
[29/200] Loss: 0.6933371424674988
[30/200] Loss: 0.693469583

In [12]:
test_set = BadmintonDataset(testing_data)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

In [13]:
# Test
ffn_model = torch.load('ffn.pt')
ffn_model.eval()

num_corrects = 0
num_total = 0
predict = []
with torch.no_grad():
    for data, label in test_loader:
        inputs = data.view(-1, 250)
        inputs = inputs.to(device).float()
        label = label.to(device).float()
        
        codes, decoded = autoencoder_model(inputs)
        output = ffn_model(codes)
        
        pred = (torch.sigmoid(output) >= 0.5).long()
        predict.extend(pred.view(-1).data.cpu().numpy())
        
        num_corrects += (pred == label).sum().item()
        num_total += len(label)

print(num_corrects / num_total)        

0.5


# Output Test

In [14]:
train_set[0]

(array([[  6.        ,   0.        ,   5.        ,   8.        ,
           0.33333333],
        [ 10.        ,   0.        ,  -1.        ,   3.        ,
           1.        ],
        [-10.        , -10.        , -10.        , -10.        ,
         -10.        ],
        [-10.        , -10.        , -10.        , -10.        ,
         -10.        ],
        [-10.        , -10.        , -10.        , -10.        ,
         -10.        ],
        [-10.        , -10.        , -10.        , -10.        ,
         -10.        ],
        [-10.        , -10.        , -10.        , -10.        ,
         -10.        ],
        [-10.        , -10.        , -10.        , -10.        ,
         -10.        ],
        [-10.        , -10.        , -10.        , -10.        ,
         -10.        ],
        [-10.        , -10.        , -10.        , -10.        ,
         -10.        ],
        [-10.        , -10.        , -10.        , -10.        ,
         -10.        ],
        [-10.        